In [16]:
%matplotlib inline
# Import some useful packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# For data preprocessing
from tensorflow.keras.utils import to_categorical

# Layers for FNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD

In [2]:
df = pd.read_csv('euw.csv')

In [3]:
df.drop(df.columns[0], axis=1, inplace=True)

In [4]:
x_train = df[df.columns[1:]]
y_train = df["result"]

In [5]:
x_train.shape

(2865, 23)

In [14]:
y_train.shape

(2865, 2)

In [6]:
y_train = to_categorical(y_train, 2)

In [7]:
y_train[1003]

array([1., 0.], dtype=float32)

In [19]:
FC_layers = [Dense(units=256, input_dim=23, activation='relu'),
             Dense(units=128, activation = 'relu'),
             Dense(units=2, activation='softmax')]

In [20]:
model = Sequential(FC_layers)
# model.add(Dense(87, input_dim=784, activation='relu'))
# model.add(Dense(128,activation = 'relu'))
# model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 256)               6144      
_________________________________________________________________
dense_12 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 258       
Total params: 39,298
Trainable params: 39,298
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(loss='mse', optimizer=SGD(lr=0.1), metrics=['accuracy'])

In [31]:
model.fit(x_train, y_train, batch_size=80, epochs=50)

Epoch 1/50
2865/2865 [==============================] - 1s 190us/sample - loss: 0.2494 - acc: 0.5215
Epoch 2/50
2865/2865 [==============================] - 0s 171us/sample - loss: 0.2495 - acc: 0.5236
Epoch 3/50
2865/2865 [==============================] - 1s 204us/sample - loss: 0.2497 - acc: 0.5106
Epoch 4/50
2865/2865 [==============================] - 0s 82us/sample - loss: 0.2494 - acc: 0.5260
Epoch 5/50
2865/2865 [==============================] - 0s 88us/sample - loss: 0.2487 - acc: 0.5215
Epoch 6/50
2865/2865 [==============================] - 0s 91us/sample - loss: 0.2494 - acc: 0.5204
Epoch 7/50
2865/2865 [==============================] - 0s 88us/sample - loss: 0.2495 - acc: 0.5075
Epoch 8/50
2865/2865 [==============================] - 0s 74us/sample - loss: 0.2490 - acc: 0.5215
Epoch 9/50
2865/2865 [==============================] - 0s 61us/sample - loss: 0.2496 - acc: 0.5197
Epoch 10/50
2865/2865 [==============================] - 0s 82us/sample - loss: 0.2496 - acc: 0.5